# LAPD Arrest and Call Data
### Call Data is from 2010, 2015, 2020 - Understanding Increases in Call Typology 

#### Call Typologies: "Party","Traffic","Racing","General Disturbance","Landlord/Tenant"

### Arrest Data is from 2010-2023 - Understanding Increases in Arrest Typology (Guided by Brenden Beck's Methodoly in "Policing Gentrification: Stops and Low–Level Arrests during Demographic Change and Real Estate Reinvestment"

#### Arrest Typologies: "Moving Violations","Weapon Possession","Narcotic Drug Laws","DUI","Disorderly Conduct","Disturbing the Peace","Drunkenness","Misc Other Violations","Prostitution","Possession of Stolen Property"

##### Calls have been filtered to show only Black arrests (continuity with Rent Burden Demographic)

In [1]:
import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import plotly.express as px
import plotly as plot
import numpy as np
import matplotlib as mpl

In [2]:
hoods = gpd.read_file('LA_Times_Neighborhood_Boundaries.geojson')

In [3]:
hoods.head()

,OBJECTID,name,geometry
0,1,Adams-Normandie,"POLYGON ((-118.30069 34.03731, -118.30388 34.0..."
1,2,Arleta,"POLYGON ((-118.44255 34.26412, -118.44240 34.2..."
2,3,Arlington Heights,"POLYGON ((-118.31862 34.05306, -118.31673 34.0..."
3,4,Atwater Village,"MULTIPOLYGON (((-118.27886 34.15321, -118.2788..."
4,5,Baldwin Hills/Crenshaw,"POLYGON ((-118.36102 34.02532, -118.36026 34.0..."


In [4]:
arrests10 = pd.read_csv('Arrest Data 2010.csv')
arrests15 = pd.read_csv('Arrest Data 2015.csv')
arrests20 = pd.read_csv('Arrest Data 2020-2023.csv')

LAPD Arrest data from 2010, 2015, and 2023-Present which has been filtered for arrests per the arrest methodoloy (Brenden Beck). 

In [5]:
blackarrests10 = arrests10[arrests10['Descent Code'] == "B"]
blackarrests15 = arrests15[arrests15['Descent Code'] == "B"]
blackarrests20 = arrests20[arrests20['Descent Code'] == "B"]

Filtering for only arrests of people identified as Black in the Descent Code category of the LAPD data

In [6]:
blackarrests10 = blackarrests10[[
    'Area Name','Age','Sex Code','Descent Code','Charge Description','LAT','LON','Location']]
blackarrests15 = blackarrests15[[
    'Area Name','Age','Sex Code','Descent Code','Charge Description','LAT','LON','Location']]
blackarrests20 = blackarrests20[[
    'Area Name','Age','Sex Code','Descent Code','Charge Description','LAT','LON','Location']]

In [28]:
hoods = hoods[['name','geometry']]

hoods.index = hoods['name']

hoodfig = px.choropleth_mapbox(hoods, geojson=hoods.geometry,locations = hoods.index,
                           color_continuous_scale="viridis",
                           mapbox_style="carto-positron",
                           zoom=6, center = {"lat": 34.0522, "lon": -118.2437},
                           opacity=0.04,)

Setting up the LA Times neighborhood map

In [8]:
df = blackarrests10
df2 = blackarrests15
df3 = blackarrests20

In [9]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.LON, df.LAT))

gdf2 = gpd.GeoDataFrame(
    df2, geometry=gpd.points_from_xy(df2.LON, df2.LAT))

gdf3 = gpd.GeoDataFrame(
    df3, geometry=gpd.points_from_xy(df3.LON, df3.LAT))

Converting the arrest point data from a Dataframe to Geodataframe this will assist later

In [10]:
gdf = gdf[['Area Name','Age','Descent Code','Sex Code','Charge Description','geometry']]
gdf2 = gdf2[['Area Name','Age','Descent Code','Sex Code','Charge Description','geometry']]
gdf3 = gdf3[['Area Name','Age','Descent Code','Sex Code','Charge Description','geometry']]

In [11]:
gdf = gdf.set_crs('epsg:4326')
gdf2 = gdf2.set_crs('epsg:4326')
gdf3 = gdf3.set_crs('epsg:4326')

With previous errors, I had learned that the arrest data did not have a set crs.  In this function, I am setting the CRS within the arrests to be the same as the LA neighborhood boundary CRS

In [12]:
join = gpd.sjoin(gdf, hoods, how="inner", predicate="within")
join1 = gpd.sjoin(gdf2, hoods, how="inner", predicate="within")
join2 = gpd.sjoin(gdf3, hoods, how="inner", predicate="within")

I have conducted a spatial join of the arrest data and the LA neighborhood boundaries.  This data will only show arrest points which have ocurred within the polygons of the neighborhood boundaries.  This will assist in later heat maps and other visualizations.  As well, this is important as some arrests points ocurred outside of LA City limits.  

In [13]:
arrests10 = join[['name','Age','Descent Code','Sex Code','Charge Description','geometry']]
arrests15 = join1[['name','Age','Descent Code','Sex Code','Charge Description','geometry']]
arrests20 = join2[['name','Age','Descent Code','Sex Code','Charge Description','geometry']]

In [29]:
arrestfig10 = px.scatter_mapbox(arrests10,
                    lat=arrests10.geometry.y,
                    lon=arrests10.geometry.x,
                    hover_name="Age", color_continuous_scale="viridis",
                           mapbox_style="carto-positron",
                           zoom=8, center = {"lat": 34.0522, "lon": -118.2437},
                           opacity=0.02)

arrest10_map = hoodfig.add_trace(
    arrestfig10.data[0])

arrest10_map.write_html("Low Level Arrests 2010.html")

In [30]:
arrestfig15 = px.scatter_mapbox(arrests15,
                    lat=arrests15.geometry.y,
                    lon=arrests15.geometry.x,
                    hover_name="Age", color_continuous_scale="viridis",
                           mapbox_style="carto-positron",
                           zoom=8, center = {"lat": 34.0522, "lon": -118.2437},
                           opacity=0.02)

arrest15_map = hoodfig.add_trace(
    arrestfig15.data[0])

arrest15_map.write_html("Low Level Arrests 2015.html")

In [31]:
arrestfig20 = px.scatter_mapbox(arrests20,
                    lat=arrests20.geometry.y,
                    lon=arrests20.geometry.x,
                    hover_name="Age", color_continuous_scale="viridis",
                           mapbox_style="carto-positron",
                           zoom=8, center = {"lat": 34.0522, "lon": -118.2437},
                           opacity=0.02)

arrest20_map = hoodfig.add_trace(
    arrestfig20.data[0])

arrest20_map.write_html("Low Level Arrests 2020.html")

These are interactive maps from 2010, 2015, 2020-Present which show the point data for low-level arrests based on our methodology.  From this data, we can begin to build out heat maps which show the total arrests in each neighborhood. 

In [18]:
arrests10.columns = ['name','Age','Descent Code','Sex Code','Charge Description','point']
arrests15.columns = ['name','Age','Descent Code','Sex Code','Charge Description','point']
arrests20.columns = ['name','Age','Descent Code','Sex Code','Charge Description','point']

In [19]:
arrests15.head()

,name,Age,Descent Code,Sex Code,Charge Description,point
8,Hollywood,23,B,M,POSS ALCH BEV ON POSTED PREM,POINT (-118.33250 34.10340)
35,Hollywood,31,B,M,VIOLATION OF PAROLE:FELONY,POINT (-118.32110 34.09980)
84,Hollywood,30,B,M,DUI ALCOHOL CAUSING INJURY,POINT (-118.32520 34.10370)
101,Hollywood,23,B,M,TRANSPORT/SELL/FURNISH/ETC MARIJUANA,POINT (-118.33870 34.10160)
190,Hollywood,32,B,M,DRUNK DRIVING ALCOHOL/DRUGS,POINT (-118.32110 34.09980)


In [20]:
arrests10group = arrests10.groupby('name').count()
arrests15group = arrests15.groupby('name').count()
arrests20group = arrests20.groupby('name').count()

In [22]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

I won't mark but I utilized the full list of each year in order to export as a csv.  This list of the spatially joined arrest data can then be remerged with the geojson of the LA neighborhoods file in order to create visualizations.

In [23]:
totals = pd.read_csv('Arrest Counts Joined.csv')

This is the separated list of totals of arrests for the spatially joined neighborhood data.  From this, I can then utilize this data to demonstrate a number of things below. 

In [46]:
df= hoods.merge(totals, left_on="name",right_on="Neighborhood", how="outer") 

In [47]:
df = df[['Neighborhood','2010','2015','2020','2010 Percent Total','2015 Percent Total','2020 Percent of Total','Percent Change 2010-2023','Burden of Arrest','geometry']]

In [27]:
df.index = df['Neighborhood']

In [34]:
arrestburden = px.choropleth_mapbox(df, geojson=df['geometry'], locations=df.index, color='Burden of Arrest',
                               mapbox_style="carto-positron",
                               range_color=(0,5),
                               zoom=9, 
                               center = {"lat": 34.0522, "lon": -118.2437},
                               opacity=0.25,
                               hover_name = df['Neighborhood'],
                               color_continuous_scale = "Viridis",)

In [35]:
arrestburden.write_html("Arrest Burden 2010-2023.html")